# Yelp reviews classification with advanced ML

## 1: Reading in the Yelp Reviews

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [3]:
yelp = pd.read_csv('yelp.csv')

In [4]:
yelp.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [5]:
# create a new DataFrame that only contains the 5-star and 1-star reviews
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]

In [6]:
# define X and y
X = yelp_best_worst.text
y = yelp_best_worst.stars

In [7]:
# split the new DataFrame into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## 2: Tokenization

- **What:** Separate text into units such as sentences or words
- **Why:** Gives structure to previously unstructured text
- **Notes:** Relatively easy with English language text, not easy with some languages

In [8]:
# use CountVectorizer to create document-term matrices from X_train and X_test
vect = CountVectorizer()

In [9]:
X_train_dtm = vect.fit_transform(X_train)

In [10]:
X_test_dtm = vect.transform(X_test)

In [11]:
# rows are documents, columns are terms (aka "tokens" or "features")
X_train_dtm.shape

(3064, 16825)

In [12]:
# last 50 features
print (vect.get_feature_names()[-50:])

['yyyyy', 'z11', 'za', 'zabba', 'zach', 'zam', 'zanella', 'zankou', 'zappos', 'zatsiki', 'zen', 'zero', 'zest', 'zexperience', 'zha', 'zhou', 'zia', 'zihuatenejo', 'zilch', 'zin', 'zinburger', 'zinburgergeist', 'zinc', 'zinfandel', 'zing', 'zip', 'zipcar', 'zipper', 'zippers', 'zipps', 'ziti', 'zoe', 'zombi', 'zombies', 'zone', 'zones', 'zoning', 'zoo', 'zoyo', 'zucca', 'zucchini', 'zuchinni', 'zumba', 'zupa', 'zuzu', 'zwiebel', 'zzed', 'éclairs', 'école', 'ém']


In [13]:
pd.DataFrame(X_train_dtm.toarray())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,16785,16786,16787,16788,16789,16790,16791,16792,16793,16794,16795,16796,16797,16798,16799,16800,16801,16802,16803,16804,16805,16806,16807,16808,16809,16810,16811,16812,16813,16814,16815,16816,16817,16818,16819,16820,16821,16822,16823,16824
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3059,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3060,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3061,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3062,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- **lowercase:** boolean, True by default
- Convert all characters to lowercase before tokenizing.

In [14]:
# don't convert to lowercase
vect = CountVectorizer(lowercase=False)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(3064, 20838)

- **ngram_range:** tuple (min_n, max_n)
- The lower and upper boundary of the range of n-values for different n-grams to be extracted. All values of n such that min_n <= n <= max_n will be used.

In [15]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 3))
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(3064, 456398)

In [16]:
# last 50 features
print (vect.get_feature_names()[-50:])

['zucchini carrots and', 'zucchini fries', 'zucchini fries are', 'zucchini pieces', 'zucchini pieces amongst', 'zucchini strips', 'zucchini strips appetizer', 'zucchini veal', 'zucchini veal demi', 'zucchini very', 'zucchini very good', 'zucchini with', 'zucchini with some', 'zuchinni', 'zuchinni again', 'zuchinni again on', 'zuchinni the', 'zuchinni the sampler', 'zumba', 'zumba class', 'zumba class and', 'zumba or', 'zumba or cycling', 'zumba yogalates', 'zumba yogalates boot', 'zupa', 'zupa flavors', 'zupa flavors are', 'zuzu', 'zuzu in', 'zuzu in downtown', 'zuzu is', 'zuzu is at', 'zuzu the', 'zuzu the ultimate', 'zwiebel', 'zwiebel kräuter', 'zwiebel kräuter salat', 'zzed', 'zzed in', 'zzed in my', 'éclairs', 'éclairs napoleons', 'éclairs napoleons and', 'école', 'école lenôtre', 'école lenôtre trained', 'ém', 'ém all', 'ém all they']


**Predicting the star rating:**

In [17]:
# use default options for CountVectorizer
#vect = CountVectorizer()
vect = CountVectorizer(ngram_range=(1, 2))

# create document-term matrices
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

# use Naive Bayes to predict the star rating
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy
print (metrics.accuracy_score(y_test, y_pred_class))
print(nb.score(X_test_dtm,y_test))

0.8542074363992173
0.8542074363992173


In [18]:
# calculate null accuracy
y_test_binary = np.where(y_test==5, 1, 0)
max(y_test_binary.mean(), 1 - y_test_binary.mean())

0.8199608610567515

In [19]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print ('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    print("Training Accuracy")
    print(nb.score(X_train_dtm,y_train))
    print("Testing Accuracy")
    print(nb.score(X_test_dtm,y_test))

In [20]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 6))
tokenize_test(vect)

Features:  1459726
Training Accuracy
0.9990208877284595
Testing Accuracy
0.8258317025440313


In [21]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 1),min_df=2)
tokenize_test(vect)

Features:  8783
Training Accuracy
0.966710182767624
Testing Accuracy
0.9246575342465754


## 3: Stopword Removal

- **What:** Remove common words that will likely appear in any text
- **Why:** They don't tell you much about your text

In [22]:
# show vectorizer options
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=2,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

- **stop_words:** string {'english'}, list, or None (default)
- If 'english', a built-in stop word list for English is used.
- If a list, that list is assumed to contain stop words, all of which will be removed from the resulting tokens.
- If None, no stop words will be used. max_df can be set to a value in the range [0.7, 1.0) to automatically detect and filter stop words based on intra corpus document frequency of terms.

In [23]:
vect = CountVectorizer(stop_words='english')

In [24]:
# set of stop words
print(vect.get_stop_words())

frozenset({'noone', 'it', 'neither', 'whenever', 'front', 'ltd', 'sincere', 'yours', 'namely', 'see', 'amount', 'ever', 'however', 'put', 'fifteen', 'elsewhere', 'since', 'the', 'them', 'to', 'two', 'no', 'further', 'cry', 'everyone', 'made', 'could', 'several', 'should', 'such', 'whereupon', 'whoever', 'already', 'found', 'out', 'anywhere', 'thence', 'his', 'before', 'and', 'nothing', 'amongst', 'became', 'almost', 'or', 'first', 'go', 'what', 'something', 'nowhere', 'all', 'am', 'besides', 'she', 'fifty', 'de', 'fire', 'over', 'ten', 'above', 'four', 'still', 'someone', 'whither', 'empty', 'mostly', 'amoungst', 'these', 'hasnt', 'else', 'again', 'move', 'top', 'below', 'everything', 'everywhere', 'seeming', 'whether', 'with', 'inc', 'nobody', 'though', 'although', 'himself', 'any', 'among', 'alone', 'hence', 'less', 'me', 'upon', 'being', 'whence', 'throughout', 'mill', 'thick', 'also', 'a', 'afterwards', 'anyway', 'during', 'he', 'interest', 'cannot', 'formerly', 'except', 'last', '

In [25]:
# remove English stop words
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  16528
Training Accuracy
0.9758485639686684
Testing Accuracy
0.9158512720156555


In [26]:
# without stopwords, dtm size
vect = CountVectorizer()
vect.fit_transform(X_train)

<3064x16825 sparse matrix of type '<class 'numpy.int64'>'
	with 237720 stored elements in Compressed Sparse Row format>

In [27]:
# with stopwords, dtm size
vect = CountVectorizer(stop_words='english')
vect.fit_transform(X_train)

<3064x16528 sparse matrix of type '<class 'numpy.int64'>'
	with 143743 stored elements in Compressed Sparse Row format>

In [28]:
my_additional_stop_words = ['abcd']

from sklearn.feature_extraction import text 
stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)

In [29]:
# with stopwords, dtm size
vect = CountVectorizer(stop_words=stop_words)
vect.fit_transform(X_train)

<3064x16528 sparse matrix of type '<class 'numpy.int64'>'
	with 143743 stored elements in Compressed Sparse Row format>

In [30]:
# remove updated stop words 
vect = CountVectorizer(stop_words=stop_words)
tokenize_test(vect)

Features:  16528
Training Accuracy
0.9758485639686684
Testing Accuracy
0.9158512720156555


## 4: Other CountVectorizer Options

- **max_features:** int or None, default=None
- If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

In [31]:
# remove English stop words and only keep 100 features
vect = CountVectorizer(stop_words='english', max_features=1)
tokenize_test(vect)

Features:  1
Training Accuracy
0.8156005221932114
Testing Accuracy
0.8199608610567515


In [32]:
# all 100 features
print (vect.get_feature_names())

['place']


In [33]:
# include 1-grams and 2-grams, and limit the number of features
vect = CountVectorizer(ngram_range=(1, 5), max_features=70000)
tokenize_test(vect)

Features:  70000
Training Accuracy
0.9918407310704961
Testing Accuracy
0.9246575342465754


- **min_df:** float in range [0.0, 1.0] or int, default=1
- When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts.

In [34]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(X_train)
    print ('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(X_test)
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    print("Training Accuracy")
    print(nb.score(X_train_dtm,y_train))
    print("Testing Accuracy")
    print(nb.score(X_test_dtm,y_test))

In [35]:
# include 1-grams and 2-grams, and only include terms that appear at least 2 times
vect = CountVectorizer(ngram_range=(1,1))
tokenize_test(vect)
#X_train_dtm = vect.fit_transform(X_train)
#X_test_dtm = vect.transform(X_test)
#nb = MultinomialNB()
#nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)
print()
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(confusion_matrix(y_test,y_pred_class))
print(classification_report(y_test,y_pred_class))

Features:  16825
Training Accuracy
0.972911227154047
Testing Accuracy
0.9187866927592955

[[ 35 149]
 [  0 838]]
              precision    recall  f1-score   support

           1       1.00      0.19      0.32       184
           5       0.85      1.00      0.92       838

    accuracy                           0.85      1022
   macro avg       0.92      0.60      0.62      1022
weighted avg       0.88      0.85      0.81      1022



In [36]:
vect = CountVectorizer(ngram_range=(1, 5), min_df=2)
tokenize_test(vect)

Features:  76347
Training Accuracy
0.9924934725848564
Testing Accuracy
0.9246575342465754
